In [1]:
from pathlib import Path
import pandas as pd
from pptx import Presentation
ROOT = Path("/Users/ylin/My Drive/Cohost/** Properties ** -- Valta/")  # <-- EDIT THIS

def find_ppts_by_subfolder(root: Path) -> pd.DataFrame:
    rows = []

    # Only immediate subfolders under ROOT (one level)
    for sub in sorted([p for p in root.iterdir() if p.is_dir() and not p.name.startswith(".")]):
        # Find ppt/pptx recursively within the subfolder
        ppts = list(sub.rglob("*.pptx")) + list(sub.rglob("*.ppt"))

        for ppt in sorted(ppts):
            if ppt.name.startswith("~$"):  # ignore Office temp files
                continue
            rows.append({
                "property_folder": sub.name,
                "ppt_name": ppt.name,
                "ppt_path": str(ppt),
            })

    return pd.DataFrame(rows).sort_values(["property_folder", "ppt_name"]).reset_index(drop=True)



In [2]:
#ppt_df = find_ppts_by_subfolder(ROOT)
#ppt_df.to_csv("/Users/ylin/My Drive/Cohost/Data and Reporting/10-OnboardingTemplate/Cohost_Property_PPTs_Locations.csv", index=False)

In [3]:
ppts_loc = pd.read_excel("/Users/ylin/My Drive/Cohost/Data and Reporting/10-OnboardingTemplate/PPT Info summary/Cohost_Property_PPTs_Locations.xlsx")
ppts_loc

,Listing,property_folder,ppt_name,ppt_path
0,Bainbridge 11143,Bainbridge 2025 - 11143 Rolling Bay Walk 98110...,Bainbridge 2025 - 11143 Rolling Bay Walk NE Ma...,/Users/ylin/My Drive/Cohost/** Properties ** -...
1,Bellevue 14615,Bellevue 2021 - 14615 NE 32nd St #D303 Bellevu...,"14615 NE 32nd St D303 Bellevue, WA 98007 Maint...",/Users/ylin/My Drive/Cohost/** Properties ** -...
2,Bellevue 14620,"Bellevue 2022 - 14620 NE 31st St #E205, 98007,...",14620 NE 31st St #E205 Maintenance Overview - ...,/Users/ylin/My Drive/Cohost/** Properties ** -...
3,Bellevue 1621,"Bellevue 2022 - 1621 107th Ave SE, Bellevue","Bellevue, 1621 107th Ave SE - Maintenance Over...",/Users/ylin/My Drive/Cohost/** Properties ** -...
4,Bellevue 514,"Bellevue 2022 - 514 142nd Ave SE APT 97, 98007",Bellevue 514 142nd Ave SE APT 97 Maintenance O...,/Users/ylin/My Drive/Cohost/** Properties ** -...
...,...,...,...,...
92,Seattle 10057 Upper,Seattle 2025-10057 17th Ave SW 98146 -Quinnlan...,Seattle 10057-UPPER 17th Ave SW 98146 Mainte...,/Users/ylin/My Drive/Cohost/** Properties ** -...
93,Seattle 10057 Lower,Seattle 2025-10057 17th Ave SW 98146 -Quinnlan...,Copy of Seattle 10057-LOWER 17th Ave SW 98146...,/Users/ylin/My Drive/Cohost/** Properties ** -...
94,Seattle 10057 Whole,Seattle 2025-10057 17th Ave SW 98146 -Quinnlan...,Seattle 10057 17th Ave SW 98146 Maintenance Ov...,/Users/ylin/My Drive/Cohost/** Properties ** -...
95,Shelton 250,"Shelton 2022 - 250 SE Dogwood Acres Rd,98584 - Ni",Shelton - 250 SE Dogwood Acres Rd Maintenance ...,/Users/ylin/My Drive/Cohost/** Properties ** -...


In [4]:
# tables = []

# for _, row in ppts_loc.iterrows():
#     ppt_path = row["ppt_path"]
#     print(f"Processing: {ppt_path}")
#     prs = Presentation(ppt_path)
#     n = min(2, len(prs.slides))  # first 2 slides, or fewer if deck is short
#     for slide_idx in range(n):
#         slide = prs.slides[slide_idx]   # integer indexing (works)
#         for shape_idx, shape in enumerate(slide.shapes, start=1):
#             if getattr(shape, "has_table", False):
#                 table = shape.table

#                 data = [
#                     [cell.text.strip() for cell in row.cells]
#                     for row in table.rows
#                 ]

#                 df = pd.DataFrame(data)
#                 df.columns = df.iloc[0]
#                 df = df[1:].reset_index(drop=True)

#                 if slide_idx == 0:
#                     df["Category"] = "Main"
#                 else:
#                     df = df.rename(columns={df.columns[2]: "Description"})
#                     df["Category"] = df["Category"].replace("", pd.NA).ffill()
        
#         df["Listing"] = row["Listing"]
#         tables.append(df)

In [5]:
#tables = pd.concat(tables, ignore_index=True, sort=False)

In [6]:
import re

_illegal_excel_re = re.compile(r"[\x00-\x08\x0B\x0C\x0E-\x1F]")

def clean_for_excel(x):
    if isinstance(x, str):
        return _illegal_excel_re.sub(" ", x)   # replace illegal chars with space
    return x

In [7]:
# out = {
#     "All_Tables": tables,
# }
# with pd.ExcelWriter(
#     "/Users/ylin/My Drive/Cohost/Data and Reporting/10-OnboardingTemplate/Property_ppt_tables.xlsx",
#     engine="openpyxl"
# ) as writer:
#     for name, df in out.items():
#         df = df.applymap(clean_for_excel)
#         df.to_excel(writer, sheet_name=name[:31], index=False)  # Excel sheet name max 31 chars

In [8]:
# merge ppt_df with master file info
property = pd.read_excel("/Users/ylin/My Drive/Cohost/Cohost Cleaner Compensation/Working/Data/Property_Cohost.xlsx")
ppt = "/Users/ylin/My Drive/Cohost/Data and Reporting/10-OnboardingTemplate/PPT Info summary/Property_ppt_tables.xlsx"
ppt_df = pd.read_excel(ppt)
ppt_df.loc[ppt_df["Listing"]=="OSBR","Listing"] = "Cottages All OSBR"
ppt_df =ppt_df.copy()

master_df= pd.read_excel("/Users/ylin/My Drive/Cohost/** Properties ** -- Valta/Listings, Team & Vendor Master Sheet.xlsx")
master_df.loc[master_df["Property"]=="Cottage 12(Caregiver)","Property"] = "Cottage 12"
master_df.loc[master_df["Property"]=="OSBR","Property"] ="Cottages All OSBR"
master_df =master_df.copy()

In [9]:
def merged_dfs(ppt: pd.DataFrame, ppt_var: list,
               master: pd.DataFrame, master_var: list,
               property: pd.DataFrame) -> pd.DataFrame:
    ppt = ppt[ppt_var]
    master = master[master_var]
    merged= (property.loc[(property["Status"]=="Active") & (property["Type"].isin(["LTR","STR"])&(~property["Listings"].isna())),["Listing","Type"]]
            .merge(ppt, left_on="Listing", right_on="Listing", how="left")
            .merge(master, left_on="Listing", right_on="Property", how="left"))
    return merged.drop(columns=["Resource","Property"]) 



In [10]:
#listing_door_code
#check_in_instruction
combined_access = merged_dfs(
    ppt=ppt_df.loc[ppt_df["Resource"].isin(["Access",'Accss & Keys info']),],
    ppt_var=["Listing","Resource","Description"],
    master=master_df,
    master_var=["Property","Access","Backup Access (default lockbox: 3012)"],
    property=property
)

combined_access.columns = ["Listing","Type","Access_ppt","Access_masterfile","Backup Access_masterfile"]

In [11]:
## listing_parking_instructions
combined = merged_dfs(
    ppt=ppt_df.loc[ppt_df["Resource"].isin(["Parking"]),],
    ppt_var=["Listing","Resource","Description"],
    master=master_df,
    master_var=["Property"],
    property=property
)
combined.columns = ["Listing","Type","Parking_ppt"]
combined_parking = combined.copy()

In [12]:
#listing_wifi_name
#listing_wifi_password

combined = merged_dfs(
    ppt=ppt_df.loc[ppt_df["Resource"].isin(["Wifi & Internet"]),],
    ppt_var=["Listing","Resource","Description"],
    master=master_df,
    master_var=["Property"],
    property=property
)
combined.columns = ["Listing","Type","Wifi_ppt"]
#combined_wifi["wifi_name"] = combined_wifi["Wifi_ppt"].strsplit()
combined_wifi = combined.copy()

In [13]:
#listing_trash_collected_on
combined = merged_dfs(
    ppt=ppt_df.loc[ppt_df["Resource"].isin(["Garbage(recommend the biggest size to avoid overflow)"]),],
    ppt_var=["Listing","Resource","Description"],
    master=master_df,
    master_var=["Property","Trash Day"],
    property=property
)
combined.columns = ["Listing","Type","Trash_ppt","Trash_master"]
combined_trash = combined.copy()

In [14]:
#critical_warning
combined = merged_dfs(
    ppt=ppt_df.loc[ppt_df["Resource"].isin(['Restrictions']),],
    ppt_var=["Listing","Resource","Description"],
    master=master_df,
    master_var=["Property"],
    property=property
)
combined.columns = ["Listing","Type","Restrictions_ppt"]
combined_warnings = combined.copy()

In [15]:
#pet_rule

combined = merged_dfs(
    ppt=ppt_df.loc[ppt_df["Resource"].isin(['Restrictions/Pets']),],
    ppt_var=["Listing","Resource","Description"],
    master=master_df,
    master_var=["Property","Pet"],
    property=property
)
combined.columns = ["Listing","Type","pets_ppt","pets_master"]
combined_pets = combined.copy()

In [ ]:
combined = {
    "listing_wifi" : combined_wifi,
    "listing_door_code": combined_access,
    "listing_parking_instructions": combined_parking,
    "listing_trash_collected_on": combined_trash,
    "critical_warning": combined_warnings,
    "pet_rule": combined_pets
}
# with pd.ExcelWriter(
#     "/Users/ylin/My Drive/Cohost/Data and Reporting/10-OnboardingTemplate/PPT Info summary/PPT_Masterfile_Combined_Info.xlsx",
#     engine="openpyxl"
# ) as writer:
#     for name, df in combined.items():
#         df.to_excel(writer, sheet_name=name[:31], index=False)  # Excel sheet name max 31 chars

In [22]:
xlsx_path = "/Users/ylin/My Drive/Cohost/Data and Reporting/10-OnboardingTemplate/PPT Info summary/"
xl = pd.ExcelFile(xlsx_path+"PPT_Masterfile_Combined_Info.xlsx")
print(xl.sheet_names)

listing_msg = pd.read_excel(xlsx_path+"listing_mgs.xlsx")
actives = property.loc[(property["Status"]=="Active") & \
                        (property["Type"].isin(["STR","LTR"])) & ~(property["Listing"].isin(["OSBR","woodinville leasing only","Bellevue 14507"])),
                       ["Listing","Type"]]
listing_msg = actives.merge(listing_msg,left_on='Listing',right_on='nickname',how='left')
listing_msg
more_tabs = {
    "listing_msg": listing_msg
}

# with pd.ExcelWriter(xlsx_path+"PPT_Masterfile_Combined_Info.xlsx", engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
#     for name, df in more_tabs.items():
#         df.to_excel(writer, sheet_name=name[:31], index=False)


['listing_wifi', 'listing_door_code', 'listing_parking_instructions', 'listing_trash_collected_on', 'critical_warning', 'pet_rule']


In [23]:
combined = (combined_parking
            .merge(combined_access.drop(columns=["Type"]),on="Listing",how="outer")
            .merge(combined_wifi.drop(columns=["Type"]),on="Listing",how="outer")
            )
combined["Valta msg"] = combined[["Parking_ppt","Access_ppt", "Wifi_ppt"]].apply(
    lambda r: "\n".join([
        f"- Parking: {r['Parking_ppt']}",
        f"- Door code: {r['Access_ppt']}",
        f"- Wifi: {r['Wifi_ppt']}"
    ]),
    axis=1
)
combined["Valta msg"]

0      - Parking: Tricky, link of video here. Max 3 c...
1      - Parking: Street parking only. Two hour signs...
2      - Parking: Street parking only. Two hour signs...
3      - Parking: Street parking only. Two hour signs...
4      - Parking: Street parking only. Two hour signs...
                             ...                        
103    - Parking: Street parking\n- Door code: Point ...
104    - Parking: Upstairs: 1-car garage on the left ...
105    - Parking: Upstairs: 1-car garage on the left ...
106    - Parking: Up to 4 cars on the parking ground,...
107    - Parking: 1 car garage connected to house wit...
Name: Valta msg, Length: 108, dtype: object

In [32]:
xl = pd.read_excel(xlsx_path+"PPT_Masterfile_Combined_Info.xlsx",sheet_name="listing_checkin")
xl = xl.merge(combined[["Listing","Valta msg"]],on="Listing",how='left')
xl["valta excel key info"] = xl['Valta msg']
xl = xl.drop(columns="Valta msg")

with pd.ExcelWriter(xlsx_path+"PPT_Masterfile_Combined_Info.xlsx", engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    # write the updated sheet
    xl.to_excel(writer, sheet_name="listing_checkin", index=False)
